# Web scraping with BeautifulSoup

Based on this excellent [article](https://www.dataquest.io/blog/web-scraping-python-using-beautiful-soup/).

In [ ]:
! pip install bs4

In [3]:
import requests
from bs4 import BeautifulSoup

## Getting the seven-day weather forecast in the US

In [ ]:
# get the HTML
page = requests.get("https://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168")

In [26]:
# parse the HTML with BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')
# find the 
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")
tonight = forecast_items[0]
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Overnight
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Overnight: Mostly clear, with a steady temperature around 60. West wind 18 to 21 mph, with gusts as high as 28 mph. " class="forecast-icon" src="newimages/medium/nfew.png" title="Overnight: Mostly clear, with a steady temperature around 60. West wind 18 to 21 mph, with gusts as high as 28 mph. "/>
 </p>
 <p class="short-desc">
  Mostly Clear
 </p>
 <p class="temp temp-low">
  Low: 60 °F
 </p>
</div>


In [ ]:
page.content

In [28]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, "html.parser")

In [ ]:
print(soup.prettify())

In [30]:
period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp").get_text()
print(period)
print(short_desc)
print(temp)

Overnight
Mostly Clear
Low: 60 °F


In [19]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['Overnight',
 'Saturday',
 'SaturdayNight',
 'Sunday',
 'SundayNight',
 'Monday',
 'MondayNight',
 'Tuesday',
 'TuesdayNight']

In [23]:
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]
print(short_descs)
print(temps)
print(descs)

['Mostly Clear', 'Sunny andBreezy', 'Mostly Clearand Breezythen MostlyCloudy', 'Mostly Sunny', 'Mostly Clearand Breezythen PartlyCloudy', 'Mostly Sunnythen Sunnyand Breezy', 'Partly Cloudyand Breezythen MostlyCloudy', 'Mostly Sunny', 'Mostly Cloudy']
['Low: 60 °F', 'High: 69 °F', 'Low: 56 °F', 'High: 74 °F', 'Low: 57 °F', 'High: 75 °F', 'Low: 57 °F', 'High: 70 °F', 'Low: 57 °F']
['Overnight: Mostly clear, with a steady temperature around 60. West wind 18 to 21 mph, with gusts as high as 28 mph. ', 'Saturday: Sunny, with a high near 69. Breezy, with a west wind 20 to 26 mph, with gusts as high as 34 mph. ', 'Saturday Night: Increasing clouds, with a low around 56. Breezy, with a west wind 20 to 25 mph decreasing to 9 to 14 mph after midnight. Winds could gust as high as 33 mph. ', 'Sunday: Mostly sunny, with a high near 74. West wind 5 to 10 mph increasing to 16 to 21 mph in the afternoon. Winds could gust as high as 26 mph. ', 'Sunday Night: Mostly clear, with a low around 57. Breezy, 

In [24]:
import pandas as pd
weather = pd.DataFrame({
    "period": periods,
    "short_desc": short_descs,
    "temp": temps,
    "desc":descs
})
weather

,period,short_desc,temp,desc
0,Overnight,Mostly Clear,Low: 60 °F,"Overnight: Mostly clear, with a steady tempera..."
1,Saturday,Sunny andBreezy,High: 69 °F,"Saturday: Sunny, with a high near 69. Breezy, ..."
2,SaturdayNight,Mostly Clearand Breezythen MostlyCloudy,Low: 56 °F,"Saturday Night: Increasing clouds, with a low ..."
3,Sunday,Mostly Sunny,High: 74 °F,"Sunday: Mostly sunny, with a high near 74. Wes..."
4,SundayNight,Mostly Clearand Breezythen PartlyCloudy,Low: 57 °F,"Sunday Night: Mostly clear, with a low around ..."
5,Monday,Mostly Sunnythen Sunnyand Breezy,High: 75 °F,"Monday: Sunny, with a high near 75. Breezy."
6,MondayNight,Partly Cloudyand Breezythen MostlyCloudy,Low: 57 °F,"Monday Night: Partly cloudy, with a low around..."
7,Tuesday,Mostly Sunny,High: 70 °F,"Tuesday: Mostly sunny, with a high near 70."
8,TuesdayNight,Mostly Cloudy,Low: 57 °F,"Tuesday Night: Mostly cloudy, with a low aroun..."


## Getting live weather updates for any location

Adapted from [this tweet](https://twitter.com/clcoding/status/1547948222978850818?t=G1tRopbPHl-sch3_z8CB2A&s=09). 

In [1]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64)\
            AppleWebKit/537.36 (KHTML, like Gecko) \
            Chrome/58.0.3029.110 Safari/537.3'}

In [4]:
city = "Graz"
request = f"{city}+weather"
res = requests.get(f"https://www.google.com/search?q={request}\
                    &oq={request}&aqs=chrome.0.35i3912j014j46j69i60.6128j1j7&sourceid=\
                    chrome&ie=UTF-8", headers=headers)

In [6]:
soup = BeautifulSoup(res.text, "html.parser")

In [9]:
location = soup.select("#wob_loc")[0].getText().strip()
time = soup.select("#wob_dts")[0].getText().strip()
info = soup.select("#wob_dc")[0].getText().strip()
weather = soup.select("#wob_tm")[0].getText().strip()

In [10]:
print(f"{location} at {time}: {info} {weather}°C")

Graz at Montag, 12:00: Sonnig 27°C
